In [1]:
import os
os.chdir('..')

In [2]:
# Leer el archivo y extraer los ejemplos  
def leer_ejemplos(archivo):  
    with open(archivo, 'r', encoding='utf-8') as file:  
        contenido = file.read()  
    ejemplos = contenido.split('EJEMPLO: ')[1:]  # Dividir por 'EJEMPLO: ' y omitir el primer elemento vacío  
    ejemplos = [ejemplo.strip() for ejemplo in ejemplos]  # Limpiar espacios en blanco al inicio y al final  
    return ejemplos  
  
# Usar la función  
preguntas_test = leer_ejemplos('docs\EJEMPLOS PREGUNTAS TÉCNICAS Y OTRAS FAQ.txt') 

In [3]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs, obtener_faqs_unir
faqs = obtener_faqs(); faqs_unir = obtener_faqs_unir()

In [4]:
from src.api.common.dependency_container import DependencyContainer
from tqdm import tqdm
import json

output1 = [];output2 = [];output3 = []; output4= []
DC = DependencyContainer(); DC.initialize()
for pregunta in tqdm(preguntas_test):
    request = {'Pregunta': pregunta, 'FAQs': faqs + faqs_unir}
    response_str = DependencyContainer.get_faqs_workflow().execute(request)
    response = json.loads(response_str)
    faq = max(response['FAQs'], key=lambda x: float(x['Score']))  
    if float(faq['Score']) >= 5: content = faq['FAQ'] 
    else: content = 'No encontramos ninguna FAQ relacionada a la pregunta.'
    output1.append(response['FAQs'][0])
    output2.append(response['FAQs'][1])
    output3.append(response['FAQs'][2])
    output4.append(content)

 98%|█████████▊| 47/48 [13:54<00:17, 17.76s/it]


JSONDecodeError: Unterminated string starting at: line 12 column 20 (char 2869)

In [10]:
import pandas
df = pandas.DataFrame()
df['Pregunta'] = preguntas_test[:-1]
df['Respuesta 1'] = output1
df['Respuesta 2'] = output2
df['Respuesta 3'] = output3
df['Respuesta final'] = output4
df.to_excel('docs/metricas_FAQs.xlsx')

In [14]:
import pandas as pd  
import plotly.express as px  
  
# Cargar datos  
df = pd.read_excel('docs/metricas_FAQs_score_7.xlsx', index_col=0)  

# Calcular las frecuencias relativas en porcentaje  
counts = df['Respuesta final'].value_counts().sort_index()  
percentages = (counts / counts.sum()) * 100  
# Crear un gráfico de barras con los porcentajes  
fig = px.bar(percentages,  
             title='Histograma de Métricas FAQs en Porcentaje',  
             labels={'index': 'Valores', 'value': 'Frecuencia (%)'},  
             template='plotly_white')  

# Mejorar visualmente el gráfico  
fig.update_layout(  
    showlegend=False,  # Ocultar la leyenda  
    xaxis_tickfont=dict(size=5),  # Reducir el tamaño de los ticks del eje X  
    xaxis_title="",  # Eliminar el título del eje X  
    yaxis=dict(  
        tickformat=".0f"  # Formato de dos decimales para los porcentajes  
    )  
)  
# Mostrar el gráfico  
fig.show()  

In [ ]:
import plotly.express as px  
import pandas as pd

df = pd.read_excel('docs/metricas_FAQS_2.xlsx', index_col=0).iloc[:,:5]

# Calcular las frecuencias relativas en porcentaje  
value_counts = df['metrica'].value_counts(normalize=True) * 100  # Esto convierte las cuentas en porcentajes  
value_counts = value_counts.sort_index()  # Asegurarse de que los índices estén ordenados para coincidir con las etiquetas personalizadas  
  
# Crear un gráfico de barras  
fig = px.bar(value_counts,  
             title='Histograma de Métricas FAQs en Porcentaje',  
             labels={'index': 'Valores', 'value': 'Frecuencia (%)'},  
             template='plotly_white')  
  
# Mejorar visualmente el gráfico  
fig.update_layout(  
    showlegend=False,  # Ocultar la leyenda  
    xaxis=dict(  
        tickmode='array',  
        tickvals=[0, 1, 2, 3],  
        ticktext=['Indeterminado', 'Malo', 'Regular', 'Bien']  
    ),  
    yaxis=dict(  
        tickformat=".0f"  # Formato de dos decimales para los porcentajes  
    )  
)  
fig.show()  

KeyError: 'metrica'